In [ ]:
class Document(object):
    def __init__(self):
        self.document_id = None  # Unique document ID
        self.title = ''  # Title of document
        self.raw_text = ''  # Holds complete text of document.
        self.terms = []  # Holds all terms.
        self.filtered_terms = []  # Holds terms without stopwords.
        self.stemmed_terms = []  # Holds terms that were stemmed with Porter algorithm. (Only relevant in PR03!)
        self.stemmed_filtered_terms = []
        # Note: See PR02 task description for instructions regarding these properties.


    def __str__(self):
        shortened_content = self.raw_text[:10] + "..." if len(self.raw_text) > 10 else self.raw_text
        return 'D' + str(self.document_id).zfill(2) + ': ' + self.title + '("' + shortened_content + '")'


In [ ]:
#halaaaaaaaaaaaaaaaa
import os
import re
def extract_collection(source_file_path: str) -> list[Document]:
    """
    Loads a text file (aesopa10.txt) and extracts each of the listed fables/stories from the file.
    :param source_file_path: File path of the file that contains the fables
    :return: List of Document objects
    """
    catalog = []  # This list will store the Document objects.

    splitter = "\n\n\n\n"
    with open(source_file_path, "r") as data_file:
        data = data_file.read().split(splitter)

        # create the subfolder
        if not os.path.exists("collection_original"):
            os.mkdir("collection_original")
        ind=0
        for index, fable in enumerate(data):
            # Exclude the unwanted sections
            if "This is the SECOND Project Gutenberg Etext of Aesop's Fables" in fable:
                continue
            if "***This edition is being officially released on March 8, 1992***" in fable:
                continue
            if "AESOP'S FABLES (82 Fables)" in fable:
                continue

            # Write each fable to a separate file
            fable_title = (
                fable.split("\n\n\n")[0]
                .lower()
                .strip()
                .strip("_")
                .replace(",", "")
                .replace(".", "")
                .replace("'", "")
                .replace(" ", "_")
            )

            # Format the file name
            file_index = str(ind).zfill(2)
            file_name = f"{file_index}_{fable_title}.txt"

            # Write the fable to a file in the 'collection_original' folder
            with open(os.path.join("collection_original", file_name), "w") as f:
                f.write(fable)

            # Create Document objects for each fable and add them to catalog
            title, *text = fable.split('\n\n\n', 1)  # Split title and text
            raw_text = fable.replace('\n', ' ')  # Full text without line breaks
            terms =extract_terms(raw_text) #raw_text.lower().split()  # Extract terms from the fable text
            parts = fable.split('\n\n\n', 1)
            title = parts[0].strip()
            text_content = parts[1] if len(parts) > 1 else ""

            document = Document()
            document.document_id = ind
            document.title = title.strip()
            document.raw_text = text_content.replace('\n', ' ')
            document.terms = terms

            catalog.append(document)
            ind=ind+1

    return catalog
def extract_terms(text):
    """
    Extract terms (words) from the given text.
    :param text: Input text
    :return: List of terms
    """
    # Replace contractions like "it's" to "it is"
    text = re.sub(r"\b([a-zA-Z]+)'s\b", r"\1 is", text)
    text = re.sub(r"[^\w\s']", "", text)  # Remove punctuation except apostrophes
    terms = text.lower().split()
    return terms

file_path = 'aesopa10.txt'
fables = extract_collection(file_path)

In [ ]:
#hala
from collections import Counter

STOP_WORD_LIST = []
def remove_symbols(text_string: str) -> str:
    """
    Removes all punctuation marks and similar symbols from a given string.
    Occurrences of "'s" are removed as well.
    :param text_string: Input text
    :return: Cleaned text
    """
    # text_string = re.sub(r"(?<!\w)'|'(?!\w)|[^\w\s']", '', text_string)
    # text_string = re.sub(r"[\n\s]+", " ", text_string).strip()
    # return text_string
    # Remove all punctuation marks except apostrophes within words and occurrences of "'s"
    text_string = re.sub(r'(?<!\w)\'|\'(?!\w)|\'s\b|[^\w\s\']', '', text_string)

    # Replace line breaks and multiple spaces with single space
    text_string = re.sub(r'[\n\s]+', ' ', text_string)

    return text_string.strip()

def is_stop_word(term: str, stop_word_list: list[str]) -> bool:
    """
    Checks if a given term is a stop word.
    :param stop_word_list: List of all considered stop words.
    :param term: The term to be checked.
    :return: True if the term is a stop word.
    """
    # TODO: Implement this function  (PR02)
    return term.lower() in stop_word_list

def remove_stop_words_from_term_list(term_list: list[str]) -> list[str]:
    """
    Takes a list of terms and removes all terms that are stop words.
    :param term_list: List that contains the terms
    :return: List of terms without stop words
    """
    STOPWORD_FILE_PATH = os.path.join('stopwords.json')
    with open(STOPWORD_FILE_PATH, "r") as json_file:
        stop_words = json.load(json_file)
    print(stop_words)
    return [term for term in term_list if not is_stop_word(term,stop_words)]

def filter_collection(collection: list[Document]):
    """
    For each document in the given collection, this method takes the term list and filters out the stop words.
    Warning: The result is NOT saved in the documents term list, but in an extra field called filtered_terms.
    :param collection: Document collection to process
    """
    for document in collection:
        cleaned_text = remove_symbols(document.raw_text).lower()
        terms = cleaned_text.split()
        document.filtered_terms = remove_stop_words_from_term_list(terms)

def load_stop_word_list(raw_file_path: str) -> list[str]:
    """
    Loads a text file that contains stop words and saves it as a list. The text file is expected to be formatted so that
    each stop word is in a new line, e. g. like englishST.txt
    :param raw_file_path: Path to the text file that contains the stop words
    :return: List of stop words
    """
    global STOP_WORD_LIST
    with open(raw_file_path, 'r') as file:
        STOP_WORD_LIST = [line.strip().lower() for line in file]
    return STOP_WORD_LIST

def create_stop_word_list_by_frequency(collection: list[Document]) -> list[str]:
    """
    Uses the method of J. C. Crouch (1990) to generate a stop word list by finding high and low frequency terms in the
    provided collection.
    :param collection: Collection to process
    :return: List of stop words
    """
    global STOP_WORD_LIST

    LOW_FREQ_THRESHOLD = 1
    HIGH_FREQ_RATIO = 0.001

    term_freq = Counter()
    total_terms = 0

    for doc in collection:
        cleaned_text = remove_symbols(doc.raw_text).lower()
        terms = cleaned_text.split()
        for term in terms:
            term_freq[term] += 1
            total_terms += 1

    high_freq_threshold = total_terms * HIGH_FREQ_RATIO
    stop_words = [term for term, freq in term_freq.items() if freq <= LOW_FREQ_THRESHOLD or freq >= high_freq_threshold]

    STOP_WORD_LIST = stop_words
    return stop_words

# def create_stop_word_list_by_frequency(collection: list[Document], low_freq_threshold=1, high_freq_ratio=0.001) -> list[str]:
#     """
#     Uses the method of J. C. Crouch (1990) to generate a stop word list by finding high and low frequency terms in the
#     provided collection.
#     :param collection: Collection to process
#     :param low_freq_threshold: The low frequency threshold
#     :param high_freq_ratio: The ratio of high frequency threshold to total terms
#     :return: List of stop words
#     """
#     term_freq = Counter()
#     total_terms = 0

#     for doc in collection:
#         cleaned_text = remove_symbols(doc.raw_text).lower()
#         terms = cleaned_text.split()
#         for term in terms:
#             term_freq[term] += 1
#             total_terms += 1

#     high_freq_threshold = total_terms * high_freq_ratio
#     stop_words = [term for term, freq in term_freq.items() if freq <= low_freq_threshold or freq >= high_freq_threshold]

#     return stop_words

# def create_stop_word_list_by_frequency(collection: list[Document]) -> list[str]:
#     """
#     Uses the method of J. C. Crouch (1990) to generate a stop word list by finding high and low frequency terms in the
#     provided collection.
#     :param collection: Collection to process
#     :return: List of stop words
#     """
#     LOW_FREQ_THRESHOLD = 1
#     HIGH_FREQ_RATIO = 0.001

#     term_freq = Counter()
#     total_terms = 0

#     for doc in collection:
#         cleaned_text = remove_symbols(doc.raw_text).lower()
#         terms = cleaned_text.split()
#         for term in terms:
#             term_freq[term] += 1
#             total_terms += 1

#     high_freq_threshold = total_terms * HIGH_FREQ_RATIO
#     stop_words = [term for term, freq in term_freq.items() if freq <= LOW_FREQ_THRESHOLD or freq >= high_freq_threshold]

#     return stop_words
def create_stop_word_list_by_frequency(collection: list[Document]) -> list[str]:
    """
    Uses the method of J. C. Crouch (1990) to generate a stop word list by finding high and low frequency terms in the
    provided collection.
    :param collection: Collection to process
    :return: List of stop words
    """
    global STOP_WORD_LIST

    # LOW_FREQ_THRESHOLD = 1
    # HIGH_FREQ_RATIO = 0.5 * len(collection)

    # term_freq = Counter()
    # total_terms = 0

    # for doc in collection:
    #     cleaned_text = remove_symbols(doc.raw_text).lower()
    #     terms = cleaned_text.split()
    #     for term in terms:
    #         term_freq[term] += 1
    #         total_terms += 1

    # high_freq_threshold = total_terms * HIGH_FREQ_RATIO
    # stop_words = [term for term, freq in term_freq.items() if freq <= LOW_FREQ_THRESHOLD or freq >= high_freq_threshold]

    # STOP_WORD_LIST = stop_words
    LOW_FREQ_THRESHOLD = 1
    HIGH_FREQ_RATIO = 0.5  # Ratio of documents in which a term appears

    term_freq = Counter()
    doc_freq = Counter()  # Document frequency

    for doc in collection:
        cleaned_text = remove_symbols(doc.raw_text).lower()
        terms = cleaned_text.split()
        unique_terms = set(terms)  # To count each term only once per document
        term_freq.update(terms)
        doc_freq.update(unique_terms)

    num_documents = len(collection)
    high_freq_threshold = num_documents * HIGH_FREQ_RATIO

    print(term_freq)
    print(doc_freq)
    print(len(term_freq))
    print(len(doc_freq))

    stop_words = [
        term for term, freq in term_freq.items()
        if freq <= LOW_FREQ_THRESHOLD or doc_freq[term] >= high_freq_threshold
    ]
    STOP_WORD_LIST = stop_words
    return stop_words



In [ ]:
stop_word_list = load_stop_word_list('englishST.txt')
print('Done.\n')

Done.



In [ ]:
import json
print('hiiiiiiiiiiiiii')
with open('stopwords.json', 'w') as f:
    json.dump(stop_word_list, f)
print("DONE Hala")

hiiiiiiiiiiiiii
DONE Hala


In [ ]:
filter_collection(fables)

['a', "a's", 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon", "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'conta

In [ ]:
# Contains all functions related to the porter stemming algorithm.
#agreed ,conflated

# hala

VOWELS = "aeiou"
CONSONANTS = "bcdfghjklmnpqrstvwxyz"


def is_consonant(word, i):
    """Check if the character at the index i in the word is a consonant."""
    if word[i] in VOWELS:
        return False
    if word[i] == 'y':
        return i == 0 or not is_consonant(word, i - 1)
    return True


def get_measure(term: str) -> int:
    """
    Returns the measure m of a given term [C](VC){m}[V].
    :param term: Given term/word
    :return: Measure value m
    """
    m = 0
    vc_sequence = False
    for i in range(len(term)):
        if is_consonant(term, i):
            if vc_sequence:
                m += 1
                vc_sequence = False
        else:
            vc_sequence = True
    return m


def condition_v(stem: str) -> bool:
    """
    Returns whether condition *v* is true for a given stem (= the stem contains a vowel).
    :param stem: Word stem to check
    :return: True if the condition *v* holds
    """
    for i in range(len(stem)):
        if stem[i] in VOWELS:
            return True
        if stem[i] == 'y' and i > 0 and is_consonant(stem, i - 1):
            return True
    return False


def condition_d(stem: str) -> bool:
    """
    Returns whether condition *d is true for a given stem (= the stem ends with a double consonant (e.g. -TT, -SS)).
    :param stem: Word stem to check
    :return: True if the condition *d holds
    """
    if len(stem) > 1 and stem[-1] == stem[-2]:
        if is_consonant(stem, len(stem) - 1):
            return True
    return False


def cond_o(stem: str) -> bool:
    """
    Returns whether condition *o is true for a given stem (= the stem ends cvc, where the second c is not W, X or Y
    (e.g. -WIL, -HOP)).
    :param stem: Word stem to check
    :return: True if the condition *o holds
    """
    if len(stem) >= 3 and is_consonant(stem, len(stem) - 3) and stem[-2] in VOWELS:
        if stem[-1] not in 'wxy' and is_consonant(stem, len(stem) - 1):
            return True
    return False


def stem_term(term: str) -> str:
    """
    Stems a given term of the English language using the Porter stemming algorithm.
    :param term:
    :return:
    """
    if term.endswith("sses"):
        term = term[:-2]
    elif term.endswith("ies"):
        term = term[:-2]
    elif term.endswith("ss"):
        pass
    elif term.endswith("s"):
        term = term[:-1]

    # Step 1b
    if term.endswith("eed"):
        if get_measure(term[:-3]) > 0:
            term = term[:-1]
    elif term.endswith("ed"):
        if condition_v(term[:-2]):
            term = term[:-2]
            if term.endswith(("at", "bl", "iz")):
                term += "e"
            elif condition_d(term) and term[-1] not in 'lsz':  # check if true
                term = term[:-1]
            elif cond_o(term) and get_measure(term) == 1:
                term += "e"
    elif term.endswith("ing"):
        if condition_v(term[:-3]):
            term = term[:-3]
            if term.endswith(("at", "bl", "iz")):
                term += "e"
            elif condition_d(term) and term[-1] not in 'lsz':  # check if true
                term = term[:-1]
            elif cond_o(term) and get_measure(term) == 1:
                term += "e"

    # Step 1c
    if term.endswith("y") and condition_v(term[:-1]):
        term = term[:-1] + "i"
    # Step 2
    pair_tests = [('ational', 'ate'), ('tional', 'tion'), ('enci', 'ence'), ('anci', 'ance'), ('izer', 'ize'),
                  ('abli', 'able'), ('alli', 'al'), ('entli', 'ent'), ('eli', 'e'), ('ousli', 'ous'),
                  ('ization', 'ize'),
                  ('ation', 'ate'), ('ator', 'ate'), ('alism', 'al'), ('iveness', 'ive'), ('fulness', 'ful'),
                  ('ousness', 'ous'), ('aliti', 'al'), ('iviti', 'ive'), ('biliti', 'ble'),('xflurti','xti')]#XFLURTI ,
    for stem, subs in pair_tests:
        if term.endswith(stem):
            if get_measure(term[:-len(stem)]) > 0:
                term = term[:-len(stem)] + subs

    # Step 3
    pair_tests2 = [('icate', 'ic'), ('ative', ''), ('alize', 'al'), ('iciti', 'ic'), ('ical', 'ic'), ('ful', ''),
                   ('ness', '')]
    for stem, subs in pair_tests2:
        if term.endswith(stem):
            if get_measure(term[:-len(stem)]) > 0:
                term = term[:-len(stem)] + subs

    suffixes = ['al', 'ance', 'ence', 'er', 'ic', 'able', 'ible', 'ant', 'ement', 'ment', 'ent', 'ou', 'ism', 'ate',
                'iti', 'ous', 'ive', 'ize']
    for suffix in suffixes:
        if term.endswith(suffix):
            if get_measure(term[:-len(suffix)]) > 1:
                term = term[:-len(suffix)]

    if get_measure(term[:-3]) > 1:
        if term.endswith('ion'):
            temp = term[:-3]
            if temp.endswith(('s', 't')):
                term = temp
    # Step 5a
    if term.endswith("e"):
        if get_measure(term[:-1]) > 1:
            term = term[:-1]
        elif get_measure(term[:-1]) == 1 and not cond_o(term[:-1]):
            term = term[:-1]

    # Step 5b
    if term.endswith("ll") and get_measure(term[:-1]) > 1:
        term = term[:-1]
    return term


def stem_all_documents(collection: list[Document]):
    """
    For each document in the given collection, this method uses the stem_term() function on all terms in its term list.
    Warning: The result is NOT saved in the document's term list, but in the extra field stemmed_terms!
    :param collection: Document collection to process
    """
    for doc in collection:
        doc.stemmed_terms = [stem_term(term) for term in doc.terms]
        if doc.filtered_terms:
            doc.stemmed_filtered_terms = [stem_term(term) for term in doc.filtered_terms]


def stem_query_terms(query: str) -> str:
    """
    Stems all terms in the provided query string.
    :param query: User query, may contain Boolean operators and spaces.
    :return: Query with stemmed terms
    """
    terms = re.split(r'(\W+)', query)
    # terms = query.split()
    stemmed_terms = [stem_term(term) for term in terms]
    return ' '.join(stemmed_terms)



In [ ]:
stem_term('electric')

'electr'

In [ ]:
stem_all_documents(fables)

In [ ]:
from nltk.stem import PorterStemmer
#troubled , relational
ps = PorterStemmer()
ps.stem('differentli')

'differ'

In [ ]:
# Contains all retrieval models.

from abc import ABC, abstractmethod
from typing import List, Dict, Set
from pyparsing import Word, alphas, infixNotation, opAssoc, ParserElement, ParseException

import re


class RetrievalModel(ABC):
    @abstractmethod
    def document_to_representation(self, document: Document, stopword_filtering=False, stemming=False):
        """
        Converts a document into its model-specific representation.
        This is an abstract method and not meant to be edited. Implement it in the subclasses!
        :param document: Document object to be represented
        :param stopword_filtering: Controls, whether the document should first be freed of stopwords
        :param stemming: Controls, whether stemming is used on the document's terms
        :return: A representation of the document. Data type and content depend on the implemented model.
        """
        raise NotImplementedError()

    @abstractmethod
    def query_to_representation(self, query: str):
        """
        Determines the representation of a query according to the model's concept.
        :param query: Search query of the user
        :return: Query representation in whatever data type or format is required by the model.
        """
        raise NotImplementedError()

    @abstractmethod
    def match(self, document_representation, query_representation) -> float:
        """
        Matches the query and document presentation according to the model's concept.
        :param document_representation: Data that describes one document
        :param query_representation:  Data that describes a query
        :return: Numerical approximation of the similarity between the query and document representation. Higher is
        "more relevant", lower is "less relevant".
        """
        raise NotImplementedError()


class LinearBooleanModel(RetrievalModel):
    def __init__(self):
        super().__init__()
        # Enable packrat parsing for better performance
        ParserElement.enablePackrat()

        # Define the grammar for parsing the query string
        self.term = Word(alphas)
        self.expr = infixNotation(
            self.term,
            [
                ('-', 1, opAssoc.RIGHT),
                ('&', 2, opAssoc.LEFT),
                ('|', 2, opAssoc.LEFT),
            ]
        )
    def document_to_representation(self, document: Document, stopword_filtering=False, stemming=False):
        if stopword_filtering == True and stemming == False:
            terms = document.filtered_terms
        elif stopword_filtering and stemming:
            terms = document.stemmed_filtered_terms
        elif stemming and stopword_filtering == False:
            terms = document.stemmed_terms
        else:
            terms = document.terms
        return set(terms)

    def query_to_representation(self, query: str):
        # Tokenize the query into words and operators
        # query = re.sub(r'(?<!\w)\'|\'(?!\w)|\'s\b|[^\w\s\']', '', query)

        # # Replace line breaks and multiple spaces with single space
        # query = re.sub(r'[\n\s]+', ' ', query)

        query = query.strip().lower()
        print(query)
        tokens = re.findall(r'\w+|[&|()-]', query)

        # Reconstruct the query with '&' where necessary
        result = []
        for i in range(len(tokens)):
            result.append(tokens[i])
            # Check if the current token and the next token are both alphanumeric, add '&' between them
            if i + 1 < len(tokens):
                if tokens[i] not in '&|()-' and tokens[i + 1] not in '&|()-':
                    result.append('&')
        # Join the tokens back into a single string representation of the query
        return ''.join(result)

    def evaluate(self, parsed, terms):
        # Check if the parsed expression is a single term (base case)
        if isinstance(parsed, str):
            # print('parsed', parsed)
            return parsed in terms # Return True if the term is in the set of relevant terms
        if parsed[0] == '-':
            return not self.evaluate(parsed[1], terms) # Negate the evaluation of the operand
        # Handle binary operators '&' (AND) and '|' (OR)
        if parsed[1] == '&':
            # Evaluate the left and right operands recursively and perform AND operation
            return self.evaluate(parsed[0], terms) and self.evaluate(parsed[2], terms)
        if parsed[1] == '|':
            # Evaluate the left and right operands recursively and perform OR operation
            return self.evaluate(parsed[0], terms) or self.evaluate(parsed[2], terms)
        # Return False if the parsed expression does not match any expected structure
        return False

    def is_relevant(self, query, terms):
        # print(query)
        # print(terms)
        try:
            # Parse the query string using the defined grammar and obtain the parsed structure
            parsed_query = self.expr.parseString(query, parseAll=True)[0]
            # Evaluate the parsed query structure against the set of relevant terms
            return self.evaluate(parsed_query, terms)
        except ParseException as pe:
            print(f"Parse error: {pe}")
            return False

    def match(self, document_representation, query_representation) -> float:
        result = self.is_relevant(query_representation, document_representation)
        return 1.0 if result else 0.0

    def __str__(self):
        return 'Boolean Model (Linear)'


class InvertedListBooleanModel(RetrievalModel):
    def __init__(self):
        super().__init__()
        self.inverted_list = None
        self.inverted_list_filtered = None
        self.inverted_list_stemmed = None
        self.inverted_list_stemmed_filtered = None

    def build_inverted_list(self, documents: List[Document], stop_word_filtering: bool, stemming: bool) -> None:
        #Builds inverted lists based on the documents provided, depending on the specified filters.
        if stemming == False and stop_word_filtering == False:
            # Initialize inverted list for unstemmed and unfiltered terms
            self.inverted_list = {}
            for doc in documents:
                terms = self.document_to_representation(doc, stop_word_filtering, stemming)
                for term in terms:
                    if term not in self.inverted_list:
                        self.inverted_list[term] = set()
                    self.inverted_list[term].add(doc.document_id)

        if stemming == False and stop_word_filtering == True:
            # Initialize inverted list for unstemmed but filtered terms
            self.inverted_list_filtered = {}
            for doc in documents:
                terms = self.document_to_representation(doc, stop_word_filtering, stemming)
                for term in terms:
                    if term not in self.inverted_list_filtered:
                        self.inverted_list_filtered[term] = set()
                    self.inverted_list_filtered[term].add(doc.document_id)

        if stemming == True and stop_word_filtering == False:
            # Initialize inverted list for stemmed but unfiltered terms
            self.inverted_list_stemmed = {}
            for doc in documents:
                terms = self.document_to_representation(doc, stop_word_filtering, stemming)
                for term in terms:
                    if term not in self.inverted_list_stemmed:
                        self.inverted_list_stemmed[term] = set()
                    self.inverted_list_stemmed[term].add(doc.document_id)

        if stemming == True and stop_word_filtering == True:
            # Initialize inverted list for stemmed and filtered terms
            self.inverted_list_stemmed_filtered = {}
            for doc in documents:
                terms = self.document_to_representation(doc, stop_word_filtering, stemming)
                for term in terms:
                    if term not in self.inverted_list_stemmed_filtered:
                        self.inverted_list_stemmed_filtered[term] = set()
                    self.inverted_list_stemmed_filtered[term].add(doc.document_id)

    def document_to_representation(self, document: Document, stopword_filtering: bool = False,
                                   stemming: bool = False) -> Set[str]:
        if stopword_filtering == True and stemming == False:
            terms = document.filtered_terms
        elif stopword_filtering and stemming:
            terms = document.stemmed_filtered_terms
        elif stemming == True and stopword_filtering == False:
            terms = document.stemmed_terms
        else:
            terms = document.terms

        return set(terms)

    def query_to_representation(self, query: str) -> List[str]:
        # return re.findall(r'[\w\-\&\|\(\)]+', query)
        # return re.findall(r'\w+|[&|()-]', query)
        # Tokenize the query into words and operators
        query = query.lower()
        terms = re.findall(r'\w+|[&|()-]', query)
        # Reconstruct the query with '&' where necessary to ensure correct boolean logic
        result = []
        i = 0
        while i < len(terms):
            if terms[i].isalnum():
                result.append(terms[i])
                # Check if the next term is also alphanumeric and not an operator
                if i + 1 < len(terms) and terms[i + 1].isalnum():
                    result.append('&')
            else:
                result.append(terms[i])
            i += 1
        # print('Q:', result)
        return result

    def match(self, document_representation, query_representation) -> float:
        # Not used in inverted list search
        pass

    def evaluate_query(self, query_terms: List[str], inv_list,documents: List[Document]) -> Set[int]:
        stack = [] # Stack to hold sets of document IDs as we evaluate the query
        operators = [] # Stack to hold operators ('&', '|', '-', '(', ')')
        # print(inv_list)
        for term in query_terms:
            if term == '&':
                # Process '&' operator: apply higher precedence operators before adding '&'
                while operators and operators[-1] in '&|':
                    self.apply_operator(stack, operators.pop(),documents)
                operators.append(term)
            elif term == '|':
                # Process '|' operator: apply higher precedence '|' operators before adding '|'
                while operators and operators[-1] == '|':
                    self.apply_operator(stack, operators.pop(),documents)
                operators.append(term)
            elif term == '-':
                operators.append(term) # Push '-' operator to the stack
            elif term == '(':
                operators.append(term) # Push '(' to the stack
            elif term == ')':
                # Process ')' operator: pop operators until matching '(' is found
                while operators and operators[-1] != '(':
                    self.apply_operator(stack, operators.pop(),documents)
                operators.pop()  # remove the '('
            else:
                # print('term :',inv_list[term])
                term_docs = inv_list.get(term, set()) # Get document IDs for the current term from inverted list
                # print('term_docs', term_docs)
                if operators and operators[-1] == '-':
                    # Handle negation '-' operator: remove documents from all available documents set
                    operators.pop()
                    all_docs = set(doc.document_id for doc in documents) # All document IDs in the collection
                    term_docs = all_docs - term_docs # Calculate the negation of term_docs
                stack.append(term_docs)
                # print(f"Added term docs for '{term}': {term_docs}")

            # print(f"Operators stack: {operators}")
            # print(f"Documents stack: {stack}")
        # After processing all terms, apply any remaining operators on the stack
        while operators:
            self.apply_operator(stack, operators.pop(),documents)
        # Return the final set of document IDs that match the query
        return stack.pop() if stack else set()


    def apply_operator(self, stack: List[Set[int]], operator: str,documents: List[Document]) -> None:
        if operator == '&':
            # Apply intersection '&' operator: Pop top two sets from stack, compute intersection, push result back
            if len(stack) > 1:
                right = stack.pop()
                left = stack.pop()
                stack.append(left & right)
        elif operator == '|':
            # Apply union '|' operator: Pop top two sets from stack, compute union, push result back
            if len(stack) > 1:
                right = stack.pop()
                left = stack.pop()
                stack.append(left | right)
                print('|: ', stack)
        elif operator == '-':
            # Apply negation '-' operator: Pop top set from stack, compute documents not in the set, push result back
            if stack:
                top = stack.pop()
                all_docs = set(d.document_id for d in documents)
                stack.append(all_docs - top)
        # print(f"Applied operator '{operator}': {stack}")

    def inverted_list_search(self, query: str, stemming: bool, stop_word_filtering: bool) -> List[tuple]:
        if stemming == False and stop_word_filtering==False:
            if self.inverted_list is None:
                self.build_inverted_list(fables, stop_word_filtering, stemming)
                query_terms = self.query_to_representation(query)
                print(query_terms)
                result_set = self.evaluate_query(query_terms,self.inverted_list,fables)

        if stemming == False and stop_word_filtering==True:
            if self.inverted_list_filtered is None:
                self.build_inverted_list(fables, stop_word_filtering, stemming)
                query_terms = self.query_to_representation(query)
                print(query_terms)
                result_set = self.evaluate_query(query_terms,self.inverted_list_filtered,fables)

        if stemming == True and stop_word_filtering==False:
            if self.inverted_list_stemmed is None:
                self.build_inverted_list(fables, stop_word_filtering, stemming)
                query_terms = self.query_to_representation(query)
                result_set = self.evaluate_query(query_terms,self.inverted_list_stemmed,fables)

        if stemming == True and stop_word_filtering==True:
            if self.inverted_list_stemmed_filtered is None:
                self.build_inverted_list(fables, stop_word_filtering, stemming)
                query_terms = self.query_to_representation(query)
                result_set = self.evaluate_query(query_terms,self.inverted_list_stemmed_filtered,fables)



        scores = [(1.0, doc) for doc in fables if doc.document_id in result_set]
        return scores


class SignatureBasedBooleanModel(RetrievalModel):
    # TODO: Implement all abstract methods. (PR04)
    def __init__(self):
        raise NotImplementedError()  # TODO: Remove this line and implement the function.

    def __str__(self):
        return 'Boolean Model (Signatures)'


class VectorSpaceModel(RetrievalModel):
    # TODO: Implement all abstract methods. (PR04)
    def __init__(self):
        raise NotImplementedError()  # TODO: Remove this line and implement the function.

    def __str__(self):
        return 'Vector Space Model'


class FuzzySetModel(RetrievalModel):
    # TODO: Implement all abstract methods. (PR04)
    def __init__(self):
        raise NotImplementedError()  # TODO: Remove this line and implement the function.

    def __str__(self):
        return 'Fuzzy Set Model'


In [ ]:
def basic_query_search( query: str, stemming: bool, stop_word_filtering: bool) -> list:
    """
    Searches the collection for a query string. This method is "basic" in that it does not use any special algorithm
    to accelerate the search. It simply calculates all representations and matches them, returning a sorted list of
    the k most relevant documents and their scores.
    :param query: Query string
    :param stemming: Controls, whether stemming is used
    :param stop_word_filtering: Controls, whether stop-words are ignored in the search
    :return: List of tuples, where the first element is the relevance score and the second the corresponding
    document
    """
    query_representation = model.query_to_representation(query)
    print(query_representation)
    document_representations = [model.document_to_representation(d, stop_word_filtering, stemming)
                                for d in fables]
    scores = [model.match(dr, query_representation) for dr in document_representations]
    ranked_collection = sorted(zip(scores, fables), key=lambda x: x[0], reverse=True)
    results = ranked_collection[:10]
    return ranked_collection

In [ ]:
    def inverted_list_search( query: str, stemming: bool, stop_word_filtering: bool) -> list:
        """
        Fast Boolean query search for inverted lists.
        :param query: Query string
        :param stemming: Controls, whether stemming is used
        :param stop_word_filtering: Controls, whether stop-words are ignored in the search
        :return: List of tuples, where the first element is the relevance score and the second the corresponding
        document
        """
        result_set=set()
        if stemming == False and stop_word_filtering == False:
            if model.inverted_list is None:
                model.build_inverted_list(fables, stop_word_filtering, stemming)
                query_terms = self.model.query_to_representation(query)
                print(query_terms)
                result_set = self.model.evaluate_query(query_terms, self.model.inverted_list,self.collection)

        if stemming == False and stop_word_filtering == True:
            if self.model.inverted_list_filtered is None:
                self.model.build_inverted_list(self.collection, stop_word_filtering, stemming)
                query_terms = self.model.query_to_representation(query)
                print(query_terms)
                result_set = self.model.evaluate_query(query_terms, self.model.inverted_list_filtered,self.collection)

        if stemming == True and stop_word_filtering == False:
            if self.model.inverted_list_stemmed is None:
                self.model.build_inverted_list(self.collection, stop_word_filtering, stemming)
                query_terms = self.model.query_to_representation(query)
                print(query_terms)
                result_set = self.model.evaluate_query(query_terms, self.model.inverted_list_stemmed,self.collection)

        if stemming == True and stop_word_filtering == True:
            if self.model.inverted_list_stemmed_filtered is None:
                self.model.build_inverted_list(self.collection, stop_word_filtering, stemming)
                query_terms = self.model.query_to_representation(query)
                result_set = self.model.evaluate_query(query_terms, self.model.inverted_list_stemmed_filtered,self.collection)

        scores = [(1.0, doc) for doc in self.collection if doc.document_id in result_set]
        return scores

In [ ]:
#hala
def read_ground_truth(file_path: str) -> Dict[str, Set[int]]:
    ground_truth = {}
    with open(file_path, 'r') as file:
        for line in file:
            # Skip lines starting with '#' or empty lines
            if line.strip() and not line.startswith('#'):
                parts = line.strip().split(' - ')
                if len(parts) == 2:
                    term, doc_ids = parts
                    # Check if doc_ids is not empty
                    if doc_ids.strip():
                        # Adjust IDs to be zero-based
                        ground_truth[term] = set(map(lambda x: int(x) - 1, doc_ids.split(',')))
                else:
                    print(f"Ignoring invalid line in ground_truth.txt: {line.strip()}")

    return ground_truth

In [ ]:
ground_truth = read_ground_truth('ground_truth.txt')

In [ ]:
import re
from collections import deque
def parse_query(query: str) -> list:
    """
    Parses the query string into tokens considering Boolean operators.
    """
    # Define operator precedence
    precedence = {'-': 3, '&': 2, '|': 1}
    output = []
    operators = deque()
    query = query.lower()
    # Tokenize the query
    tokens = re.findall(r'\b\w+\b|[\-&|()]', query)

    for token in tokens:
        if token.isalnum():
            output.append(token)
        elif token in precedence:
            while (operators and operators[-1] != '(' and
                   precedence.get(token, 0) <= precedence.get(operators[-1], 0)):
                output.append(operators.pop())
            operators.append(token)
        elif token == '(':
            operators.append(token)
        elif token == ')':
            while operators and operators[-1] != '(':
                output.append(operators.pop())
            operators.pop()  # Pop '('

    while operators:
        output.append(operators.pop())

    return output
def evaluate_query(tokens: list, ground_truth: dict) -> set:
    stack = []
    for token in tokens:
        if token.isalnum():
            stack.append(ground_truth.get(token, set()))
        elif token == '&':
            right = stack.pop()
            left = stack.pop()
            stack.append(left & right)
        elif token == '|':
            right = stack.pop()
            left = stack.pop()
            stack.append(left | right)
        elif token == '-':
            operand = stack.pop()
            stack.append(ground_truth['ALL_DOCS'] - operand)
    return stack[0] if stack else set()

def calculate_precision(query: str, result_list: list[tuple]) -> float:
    ground_truth = read_ground_truth('ground_truth.txt')
    ground_truth['ALL_DOCS'] = set(doc.document_id for score, doc in result_list)

    tokens = parse_query(query)
    print('tokens',tokens)
    terms = [term for term in tokens if term.isalnum()]
    if not all(term in ground_truth for term in terms):
        return -1
    relevant_ground_truth = evaluate_query(tokens, ground_truth)
    print('relevant_ground_truth',relevant_ground_truth)
    relevant_docs = set(doc.document_id for score, doc in result_list if score >0)
    if len(relevant_docs) == 0:
        return -1
    print(len(relevant_docs & relevant_ground_truth))
    print(len(relevant_docs))
    if not relevant_docs:
        return -1
    if not relevant_ground_truth:
        return -1  # Precision calculation not possible

    precision = len(relevant_docs & relevant_ground_truth) / len(relevant_docs)
    return precision

def calculate_recall(query: str, result_list: list[tuple]) -> float:
    ground_truth['ALL_DOCS'] = set(doc.document_id for score, doc in result_list)
    query_terms = parse_query(query)
    terms = [term for term in query_terms if term.isalnum()]
    if not all(term in ground_truth for term in terms):
        return -1
    relevant_ground_truth = evaluate_query(query_terms, ground_truth)
    print('relevant_ground_truth',relevant_ground_truth)
    relevant_docs = set(doc.document_id for score, doc in result_list if score> 0 )
    if len(relevant_docs) == 0:
        return -1
    print(len(relevant_docs & relevant_ground_truth))
    print(len(relevant_ground_truth))
    if not relevant_docs:
        return -1
    if not relevant_ground_truth:
        return -1  # Precision calculation not possible
    recall = len(relevant_docs & relevant_ground_truth) / len(relevant_ground_truth)
    return recall

In [ ]:
def calculate_precision(query: str, result_list: list[tuple]) -> float:
    # TODO: Implement this function (PR03)
    # ground_truth = read_ground_truth('raw_data/ground_truth.txt')
    query_terms = re.findall(r'\b\w+\b', query)
    # print(query_terms)
    terms = [term for term in query_terms if term.isalnum()]
    # print(terms)

    if not all(term in ground_truth for term in terms):
        return -1
    relevant_docs = set()
    relevant_docs = set(doc.document_id for score, doc in result_list if score == 1)

    if len(relevant_docs) == 0:
        return -1

    relevant_ground_truth = set()
    for term in ground_truth.keys():
        if term in terms:
            relevant_ground_truth |= ground_truth[term]
            # print(relevant_ground_truth)

    if not relevant_ground_truth:
        return -1  # Precision calculation not possible

    precision = len(relevant_docs & relevant_ground_truth) / len(relevant_docs)
    return precision

def calculate_recall(query: str, result_list: list[tuple]) -> float:
    # TODO: Implement this function (PR03)
    # ground_truth = self.read_ground_truth('raw_data/ground_truth.txt')
    query_terms2 = re.findall(r'\b\w+\b', query)
    terms = [term for term in query_terms2 if term.isalnum()]
    if not all(term in ground_truth for term in terms):
        return -1
    relevant_docs = set(doc.document_id for score, doc in result_list if score == 1)
    if len(relevant_docs) == 0:
        return -1
    relevant_ground_truth = set()
    for term in ground_truth.keys():
        if term in terms:
            relevant_ground_truth |= ground_truth[term]

    if not relevant_ground_truth:
        return -1  # Recall calculation not possible

    recall = len(relevant_docs & relevant_ground_truth) / len(relevant_ground_truth)
    return recall

In [ ]:
query = input('Query: ')
query_s = stem_query_terms(query)
model = LinearBooleanModel()
results = basic_query_search(query_s, stemming=True, stop_word_filtering= False)

Query: -(fox|beast)
-( fox | beast )
-(fox|beast)


In [ ]:
print(f'precision: {calculate_precision(query,results)}')
print(f'recall: {calculate_recall(query,results)}')

tokens ['fox', 'beast', '|', '-']
relevant_ground_truth {0, 1, 2, 4, 5, 6, 8, 12, 13, 15, 16, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 49, 50, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80}
58
62
precision: 0.9354838709677419
relevant_ground_truth {0, 1, 2, 4, 5, 6, 8, 12, 13, 15, 16, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 49, 50, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80}
58
60
recall: 0.9666666666666667


In [ ]:
for score, doc in results:
    print(f"Score: {score}, Document: {doc}")

Score: 1.0, Document: D00: The Cock and the Pearl("A cock was...")
Score: 1.0, Document: D01: The Wolf and the Lamb("Once upon ...")
Score: 1.0, Document: D02: The Dog and the Shadow("It happene...")
Score: 1.0, Document: D04: The Wolf and the Crane("A Wolf had...")
Score: 1.0, Document: D05: The Man and the Serpent("A Countrym...")
Score: 1.0, Document: D06: The Town Mouse and the Country Mouse("Now you mu...")
Score: 1.0, Document: D08: The Sick Lion("A Lion had...")
Score: 1.0, Document: D11: The Swallow and the Other Birds("It happene...")
Score: 1.0, Document: D12: The Frogs Desiring a King("The Frogs ...")
Score: 1.0, Document: D13: The Mountains in Labour("One day th...")
Score: 1.0, Document: D15: The Wolf and the Kid("A Kid was ...")
Score: 1.0, Document: D16: The Woodman and the Serpent("One wintry...")
Score: 1.0, Document: D17: The Bald Man and the Fly("There was ...")
Score: 1.0, Document: D20: The Jay and the Peacock("A Jay vent...")
Score: 1.0, Document: D21: The Frog an

In [ ]:
for score, doc in results:
    print(f"Score: {score}, Document: {doc}")

Score: 1.0, Document: D00: The Cock and the Pearl("A cock was...")
Score: 1.0, Document: D01: The Wolf and the Lamb("Once upon ...")
Score: 1.0, Document: D02: The Dog and the Shadow("It happene...")
Score: 1.0, Document: D04: The Wolf and the Crane("A Wolf had...")
Score: 1.0, Document: D05: The Man and the Serpent("A Countrym...")
Score: 1.0, Document: D06: The Town Mouse and the Country Mouse("Now you mu...")
Score: 1.0, Document: D08: The Sick Lion("A Lion had...")
Score: 1.0, Document: D11: The Swallow and the Other Birds("It happene...")
Score: 1.0, Document: D12: The Frogs Desiring a King("The Frogs ...")
Score: 1.0, Document: D13: The Mountains in Labour("One day th...")
Score: 1.0, Document: D15: The Wolf and the Kid("A Kid was ...")
Score: 1.0, Document: D16: The Woodman and the Serpent("One wintry...")
Score: 1.0, Document: D17: The Bald Man and the Fly("There was ...")
Score: 1.0, Document: D20: The Jay and the Peacock("A Jay vent...")
Score: 1.0, Document: D21: The Frog an

In [ ]:
print(f'precision: {calculate_precision(query,results)}')
print(f'recall: {calculate_recall(query,results)}')

precision: 1.0
recall: 0.0


In [ ]:

query = input('Query: ')
query_s = stem_query_terms(query)
model = InvertedListBooleanModel()
results = model.inverted_list_search(query, stemming=True, stop_word_filtering=False)
for score, doc in results:
    print(f"Score: {score}, Document: {doc}")

Query: -fox
Score: 1.0, Document: D00: The Cock and the Pearl("A cock was...")
Score: 1.0, Document: D01: The Wolf and the Lamb("Once upon ...")
Score: 1.0, Document: D02: The Dog and the Shadow("It happene...")
Score: 1.0, Document: D04: The Wolf and the Crane("A Wolf had...")
Score: 1.0, Document: D05: The Man and the Serpent("A Countrym...")
Score: 1.0, Document: D06: The Town Mouse and the Country Mouse("Now you mu...")
Score: 1.0, Document: D08: The Sick Lion("A Lion had...")
Score: 1.0, Document: D09: The Ass and the Lapdog("A Farmer o...")
Score: 1.0, Document: D10: The Lion and the Mouse("Once when ...")
Score: 1.0, Document: D11: The Swallow and the Other Birds("It happene...")
Score: 1.0, Document: D12: The Frogs Desiring a King("The Frogs ...")
Score: 1.0, Document: D13: The Mountains in Labour("One day th...")
Score: 1.0, Document: D14: The Hares and the Frogs("The Hares ...")
Score: 1.0, Document: D15: The Wolf and the Kid("A Kid was ...")
Score: 1.0, Document: D16: The Wo

In [ ]:
print(f'precision: {calculate_precision(query,results)}')
print(f'recall: {calculate_recall(query,results)}')

tokens ['fox', '-']
relevant_ground_truth {0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80}
68
68
precision: 1.0
relevant_ground_truth {0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80}
68
68
recall: 1.0


In [ ]:
pip install bitarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import hashlib
from abc import ABC, abstractmethod
#hii
class SignatureBasedBooleanModel(RetrievalModel):
    def __init__(self):
        self.bit_vector_size = 64  # Signature length F
        self.num_bits = 7  # Signature weight m
        self.block_size = 4  # Overlay factor D
        self.documents = []
        self.term_dict = {}
        self.word_signatures = {}

    def __str__(self):
        return 'Boolean Model (Signatures)'

    def hash_term(self, term):
        bit_positions = set()
        seed = 0
        while len(bit_positions) < self.num_bits:
            hash_value = int(hashlib.sha256((term + str(seed)).encode()).hexdigest(), 16)
            bit_position = hash_value % self.bit_vector_size
            bit_positions.add(bit_position)
            seed += 1
        return bit_positions

    def create_word_signature(self, term):
        bit_positions = self.hash_term(term)
        signature = [0] * self.bit_vector_size
        for pos in bit_positions:
            signature[pos] = 1
        return signature

    def combine_signatures(self, signatures):
        # print('block signature')
        # print(signatures)
        combined = signatures[0][:]
        # print('combined')
        # print(combined)
        for sig in signatures[1:]:
            combined = [c | s for c, s in zip(combined, sig)]
        # print('after')
        # print(combined)
        return combined

    def create_block_signatures(self, terms):
        blocks = [terms[i:i + self.block_size] for i in range(0, len(terms), self.block_size)]
        # print("blocks")
        # print(blocks)
        block_signatures = []
        for block in blocks:
            block_sigs = [self.word_signatures[term] for term in block if term in self.word_signatures]
            if block_sigs:
                block_signature = self.combine_signatures(block_sigs)
                block_signatures.append(block_signature)
        return block_signatures

    def document_to_representation(self, document: Document, stopword_filtering=False, stemming=False):
        terms = document.terms
        if stopword_filtering:
            terms = document.filtered_terms
        if stemming:
            terms = document.stemmed_terms
        self.documents.append(document)
        for term in terms:
            # print(term)
            if term not in self.word_signatures:
                self.word_signatures[term] = self.create_word_signature(term)
            # print(self.word_signatures[term])
        document_representation = self.create_block_signatures(terms)
        # print(document_representation)
        return document_representation

    def query_to_representation(self, query: str):
        terms = query.lower().split()
        query_signatures = {}
        for term in terms:
          if term not in query_signatures:
            query_signatures[term] = self.create_word_signature(term)
            # print(query_signatures[term])
        query_sigs = [query_signatures[term] for term in terms if term in query_signatures]
        if query_sigs:
            return self.combine_signatures(query_sigs)
        return [0] * self.bit_vector_size

    def match(self, document_representation, query_representation) -> float:
        return any(self.compare_signatures(block_signature, query_representation) for block_signature in document_representation)

    def compare_signatures(self, block_signature, query_signature):
        # print(block_signature)
        # print(query_signature)
        # print(all(q <= b for q, b in zip(query_signature, block_signature)))
        return all(q <= b for q, b in zip(query_signature, block_signature))

    def string_matching(self, document, query_terms):
        # Implement a detailed string matching logic here
        document_text = ' '.join(document.terms).lower()
        return all(term in document_text for term in query_terms)

    # def parse_query(self, query: str):
    #     or_terms = re.split(r'\s*\|\s*', query)
    #     parsed_query = []
    #     for or_term in or_terms:
    #         and_terms = re.split(r'\s*&\s*', or_term)
    #         parsed_query.append(and_terms)
    #     return parsed_query
    def parse_query(self, query: str):
        query = query.replace(' ', ' & ')
        tokens = re.split(r'(\W)', query)
        tokens = [token for token in tokens if token.strip() != '']

        stack = []
        current = []
        for token in tokens:
            if token == '(':
                stack.append(current)
                current = []
            elif token == ')':
                last = current
                current = stack.pop()
                current.append(last)
            elif token == '|':
                current.append('|')
            elif token == '&':
                current.append('&')
            else:
                current.append(token)

        return current

    def process_query_terms(self, query_terms):
        if isinstance(query_terms, str):
            return [[query_terms]]
        if '|' in query_terms:
            or_index = query_terms.index('|')
            left = query_terms[:or_index]
            right = query_terms[or_index + 1:]
            return self.process_query_terms(left) + self.process_query_terms(right)
        if '&' in query_terms:
            and_index = query_terms.index('&')
            left = query_terms[:and_index]
            right = query_terms[and_index + 1:]
            left_processed = self.process_query_terms(left)
            right_processed = self.process_query_terms(right)
            combined = []
            for l in left_processed:
                for r in right_processed:
                    combined.append(l + r)
            return combined
        if isinstance(query_terms, list):
            if len(query_terms) == 1:
                return self.process_query_terms(query_terms[0])
            combined = []
            for term in query_terms:
                processed = self.process_query_terms(term)
                for p in processed:
                    combined.append(p)
            return combined

    def signature_search(self, query: str, stemming: bool, stop_word_filtering: bool) -> list:
        parsed_query = self.parse_query(query)
        print(parsed_query)
        processed_terms = self.process_query_terms(parsed_query)
        print(processed_terms)
        query_representation = self.query_to_representation(query)
        # print(query_representation)

        results = []
        for document in fables:
            document_representation = self.document_to_representation(document, stop_word_filtering, stemming)
            if self.match(document_representation, query_representation):
                # print(document_representation)
                if self.string_matching(document, query.lower().split()):
                    results.append((1.0, document))
                # results.append((1.0, document))  # Assuming a simple match/no match scoring
        # print(self.word_signatures)
        return results

In [ ]:
query = input('Query: ')
query_s = stem_query_terms(query)
model = SignatureBasedBooleanModel()
results = model.signature_search(query, stemming=True, stop_word_filtering=True)
for score, doc in results:
    print(f"Score: {score}, Document: {doc}")

Query: fox&beast|lion
['fox', '&', 'beast', '|', 'lion']
[['fox', 'beast'], ['lion']]


In [ ]:
import hashlib
from abc import ABC, abstractmethod
#hii
class SignatureBasedBooleanModel(RetrievalModel):
    def __init__(self):
        self.bit_vector_size = 64  # Signature length F
        self.num_bits = 7  # Signature weight m
        self.block_size = 4  # Overlay factor D
        self.documents = []
        self.term_dict = {}
        self.word_signatures = None
        self.word_signatures_filtered = None
        self.word_signatures_stemmed = None
        self.word_signatures_filtered_stemmed = None
        self.document_block_signatures = {}
        self.document_block_signatures_filtered = {}
        self.document_block_signatures_stemmed = {}
        self.document_block_signatures_filtered_stemmed = {}

    def __str__(self):
        return 'Boolean Model (Signatures)'

    def hash_term(self, term):
        bit_positions = set()
        seed = 0
        while len(bit_positions) < self.num_bits:
            hash_value = int(hashlib.sha256((term + str(seed)).encode()).hexdigest(), 16)
            bit_position = hash_value % self.bit_vector_size
            bit_positions.add(bit_position)
            seed += 1
        return bit_positions

    def create_word_signature(self, term):
        bit_positions = self.hash_term(term)
        signature = [0] * self.bit_vector_size
        for pos in bit_positions:
            signature[pos] = 1
        return signature

    def combine_signatures(self, signatures):
        # print('block signature')
        # print(signatures)
        combined = signatures[0][:]
        # print('combined')
        # print(combined)
        for sig in signatures[1:]:
            combined = [c | s for c, s in zip(combined, sig)]
        # print('after')
        # print(combined)
        return combined

    def create_block_signatures(self, terms,term_sig):
        blocks = [terms[i:i + self.block_size] for i in range(0, len(terms), self.block_size)]
        # print("blocks")
        # print(blocks)
        block_signatures = []
        for block in blocks:
            block_sigs = [term_sig[term] for term in block if term in term_sig]
            if block_sigs:
                block_signature = self.combine_signatures(block_sigs)
                block_signatures.append(block_signature)
        return block_signatures

    def build_term_signatures(self, documents: List[Document], stop_word_filtering=False, stemming=False):
        if stemming == False and stop_word_filtering == False:
            # Initialize inverted list for unstemmed and unfiltered terms
            if self.word_signatures is None:
              self.word_signatures = {}
              for doc in documents:
                  terms = self.get_terms(doc, stop_word_filtering, stemming)
                  for term in terms:
                      if term not in self.self.word_signatures:
                          self.word_signatures[term] = self.create_word_signature(term)
                  # document_representation = self.create_block_signatures(terms)
                  block_signatures = self.create_block_signatures(terms,self.word_signatures)
                  self.document_block_signatures[doc.document_id] = block_signatures

        if stemming == False and stop_word_filtering == True:
            # Initialize inverted list for unstemmed but filtered terms
            if self.word_signatures_filtered is None:
                self.word_signatures_filtered = {}
                for doc in documents:
                    terms = self.get_terms(doc, stop_word_filtering, stemming)
                    for term in terms:
                        if term not in self.word_signatures_filtered:
                            self.word_signatures_filtered[term] = self.create_word_signature(term)
                    block_signatures = self.create_block_signatures(terms,self.word_signatures_filtered)
                    self.document_block_signatures_filtered[doc.document_id] = block_signatures

        if stemming == True and stop_word_filtering == False:
            # Initialize inverted list for stemmed but unfiltered terms
            if self.word_signatures_stemmed is None:
                self.word_signatures_stemmed = {}
                for doc in documents:
                    terms = self.get_terms(doc, stop_word_filtering, stemming)
                    for term in terms:
                        if term not in self.word_signatures_stemmed:
                            self.word_signatures_stemmed[term] = self.create_word_signature(term)
                    block_signatures = self.create_block_signatures(terms,self.word_signatures_stemmed)
                    self.document_block_signatures_stemmed[doc.document_id] = block_signatures

        if stemming == True and stop_word_filtering == True:
            # Initialize inverted list for stemmed and filtered terms
            if self.word_signatures_filtered_stemmed is None:
                self.word_signatures_filtered_stemmed = {}
                for doc in documents:
                    terms = self.get_terms(doc, stop_word_filtering, stemming)
                    for term in terms:
                        if term not in self.word_signatures_filtered_stemmed:
                            self.word_signatures_filtered_stemmed[term] = self.create_word_signature(term)
                    block_signatures = self.create_block_signatures(terms,self.word_signatures_filtered_stemmed)
                    self.document_block_signatures_filtered_stemmed[doc.document_id] = block_signatures


    def get_terms(self, document: Document, stopword_filtering=False, stemming=False):
        terms = document.terms
        if stopword_filtering:
            terms = document.filtered_terms
        if stemming:
            terms = document.stemmed_terms
        # self.documents.append(document)
        # for term in terms:
        #     # print(term)
        #     if term not in self.word_signatures:
        #         self.word_signatures[term] = self.create_word_signature(term)
        #     # print(self.word_signatures[term])
        # document_representation = self.create_block_signatures(terms)
        # print(document_representation)
        # return document_representation
        return terms
    def query_to_representation(self, query: str):
        pass
    def document_to_representation(self, document: Document, stop_word_filtering=False, stemming=False):
        if stemming == False and stop_word_filtering == False:
            # if document.document_id in self.document_block_signatures:
                return self.document_block_signatures[document.document_id]
        elif stemming == False and stop_word_filtering == True:
            # if document.document_id in self.document_block_signatures_filtered:
                return self.document_block_signatures_filtered[document.document_id]
        elif stemming == True and stop_word_filtering == False:
                return self.document_block_signatures_stemmed[document.document_id]
        elif stemming == True and stop_word_filtering == True:
                return self.document_block_signatures_filtered_stemmed[document.document_id]


    # def query_to_representation(self, query: str):
    #     terms = query.lower().split()
    #     query_signatures = {}
    #     for term in terms:
    #       if term not in query_signatures:
    #         query_signatures[term] = self.create_word_signature(term)
    #         # print(query_signatures[term])
    #     query_sigs = [query_signatures[term] for term in terms if term in query_signatures]
    #     if query_sigs:
    #         return self.combine_signatures(query_sigs)
    #     return [0] * self.bit_vector_size





    def match(self, document_representation, query_representation) -> float:
        return any(self.compare_signatures(block_signature, query_representation) for block_signature in document_representation)

    def compare_signatures(self, block_signature, query_signature):
        # print(block_signature)
        # print(query_signature)
        # print(all(q <= b for q, b in zip(query_signature, block_signature)))
        return all(q <= b for q, b in zip(query_signature, block_signature))

    def string_matching(self, document, term_groups, stop_word_filtering=False, stemming=False):
        # Implement a detailed string matching logic here
        # document_text = ' '.join(document.terms).lower()
        # return all(term in document_text for term in query_terms)
        if stemming == False and stop_word_filtering == False:
            # if document.document_id in self.document_block_signatures:
                doc_terms = document.terms
        elif stemming == False and stop_word_filtering == True:
            doc_terms = document.filtered_terms
        elif stemming == True and stop_word_filtering == False:
                doc_terms = document.stemmed_terms
        elif stemming == True and stop_word_filtering == True:
                doc_terms = document.stemmed_filtered_terms
        print(term_groups)
        print(doc_terms)
        for group in term_groups:
          if all(term in doc_terms for term in group):
              return True
        return False

    # def parse_query(self, query: str):
    #     or_terms = re.split(r'\s*\|\s*', query)
    #     parsed_query = []
    #     for or_term in or_terms:
    #         and_terms = re.split(r'\s*&\s*', or_term)
    #         parsed_query.append(and_terms)
    #     return parsed_query
    import re

    def parse_query(self, query: str):
        # Split query using regex to preserve operators and parentheses
        query = query.lower()
        tokens = re.findall(r'\(|\)|&|\||\w+', query)

        # Clean up and filter tokens
        tokens = [token.strip() for token in tokens if token.strip()]
        print('tokens :',tokens)
        result = []
        for i in range(len(tokens)):
            result.append(tokens[i])
            # Check if the current token and the next token are both alphanumeric, add '&' between them
            if i + 1 < len(tokens):
                if tokens[i] not in '&|()-' and tokens[i + 1] not in '&|()-':
                    result.append('&')
        # Join the tokens back into a single string representation of the query
        tokens = ''.join(result)
        tokens = re.findall(r'\(|\)|&|\||\w+', tokens)

        # Clean up and filter tokens
        # tokens = [token.strip() for token in tokens if token.strip()]
        print('tokens :',tokens)
        stack = []
        current = []
        for token in tokens:
            if token == '(':
                stack.append(current)
                current = []
            elif token == ')':
                last = current
                current = stack.pop()
                current.append(last)
            elif token == '|':
                current.append('|')
            elif token == '&':
                current.append('&')
            else:
                current.append(token)

        return current


    def process_query_terms(self, query_terms):
        if isinstance(query_terms, str):
            return [[query_terms]]
        if '|' in query_terms:
            or_index = query_terms.index('|')
            left = query_terms[:or_index]
            right = query_terms[or_index + 1:]
            return self.process_query_terms(left) + self.process_query_terms(right)
        if '&' in query_terms:
            and_index = query_terms.index('&')
            left = query_terms[:and_index]
            right = query_terms[and_index + 1:]
            left_processed = self.process_query_terms(left)
            right_processed = self.process_query_terms(right)
            combined = []
            for l in left_processed:
                for r in right_processed:
                    combined.append(l + r)
            return combined
        if isinstance(query_terms, list):
            if len(query_terms) == 1:
                return self.process_query_terms(query_terms[0])
            combined = []
            for term in query_terms:
                processed = self.process_query_terms(term)
                for p in processed:
                    combined.append(p)
            return combined

    # def signature_search(self, query: str, stemming: bool, stop_word_filtering: bool) -> list:
    #     # parsed_query = self.parse_query(query)
    #     # print(parsed_query)
    #     # processed_terms = self.process_query_terms(parsed_query)
    #     # print(processed_terms)
    #     self.build_term_signatures(fables,stop_word_filtering,stemming)
    #     # print(self.document_block_signatures_filtered_stemmed)
    #     query_representation = self.query_to_representations___test(query,stop_word_filtering,stemming)
    #     print(query_representation)

    #     results = []
    #     for document in fables:
    #         document_representation = self.document_to_representation(document, stop_word_filtering, stemming)
    #         if self.match(document_representation, query_representation):
    #             # print(document_representation)
    #             if self.string_matching(document, query.lower().split()):
    #                 results.append((1.0, document))
    #             # results.append((1.0, document))  # Assuming a simple match/no match scoring
    #     # print(self.word_signatures)
    #     return results

    def signature_search(self, query: str, stemming: bool, stop_word_filtering: bool) -> list:
        self.build_term_signatures(fables, stop_word_filtering, stemming)
        query_signatures , term_groups = self.query_to_representations___test(query, stop_word_filtering, stemming)
        print(query_signatures)
        print(term_groups)

        results = []
        for document in fables:
            document_representation = self.document_to_representation(document, stop_word_filtering, stemming)

            if any(self.match(document_representation, query_sig) for query_sig in query_signatures):
                print('halaaaaaaaaaaaaaaaaaaaaaa')
                print(document.document_id)
                if self.string_matching(document, term_groups,stop_word_filtering,stemming):
                    results.append((1.0, document))

        return results

    def query_to_representations___test(self, query: str, stop_word_filtering=False, stemming=False):
        parsed_query = self.parse_query(query)
        parsed_query = self.process_query_terms(parsed_query)
        query_signatures = []
        sig_test={}
        if stemming == False and stop_word_filtering == False:
           sig_test = self.word_signatures
        elif stemming == False and stop_word_filtering == True:
           sig_test = self.word_signatures_filtered
        elif stemming == True and stop_word_filtering == False:
           sig_test = self.word_signatures_stemmed
        elif stemming == True and stop_word_filtering == True:
           sig_test = self.word_signatures_filtered_stemmed
        for term_group in parsed_query:
            print(term_group)
            print('------------')
            if isinstance(term_group, str):  # Single term case
                term_group = [term_group]
                print('hiiii')
                print(term_group)

            group_signature = None
            for term in term_group:
                print('tem_group',term_group)
                term = term.lower()
                if term in sig_test:
                    if group_signature is None:
                        group_signature = sig_test[term][:]
                    else:
                        group_signature = [g | s for g, s in zip(group_signature, sig_test[term])]

            if group_signature:
                print('group_signature',group_signature)
                query_signatures.append(group_signature)

        if query_signatures:
            # Union of all group signatures
            query_representation = query_signatures[0]
            print('query_signatures[0]',query_signatures[0])
            for signature in query_signatures[1:]:
                query_representation = [q | s for q, s in zip(query_representation, signature)]
        else:
            query_representation = [0] * self.bit_vector_size

        return query_signatures, parsed_query

In [ ]:
query = input('Query: ')
query_s = stem_query_terms(query)
print(query_s)
model = SignatureBasedBooleanModel()
results = model.signature_search(query_s, stemming=True, stop_word_filtering=True)
for score, doc in results:
    print(f"Score: {score}, Document: {doc}")

# (fox beast )|lion

Query: fox&beast|lion
fox & beast | lion
tokens : ['fox', '&', 'beast', '|', 'lion']
tokens : ['fox', '&', 'beast', '|', 'lion']
['fox', 'beast']
------------
tem_group ['fox', 'beast']
tem_group ['fox', 'beast']
group_signature [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
['lion']
------------
tem_group ['lion']
group_signature [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
query_signatures[0] [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
[[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(f'precision: {calculate_precision(query,results)}')
print(f'recall: {calculate_recall(query,results)}')

tokens ['beast']
relevant_ground_truth {33, 3, 72, 9, 10, 11, 42, 14, 17, 51, 23, 58}
8
10
precision: 0.8
relevant_ground_truth {33, 3, 72, 9, 10, 11, 42, 14, 17, 51, 23, 58}
8
12
recall: 0.6666666666666666


In [ ]:
print(f'precision: {calculate_precision("fox beast",results)}')
print(f'recall: {calculate_recall("fox beast",results)}')

tokens ['fox', 'beast']
relevant_ground_truth {64, 33, 3, 37, 7, 72, 73, 48, 81, 18, 19, 58, 30, 63}
14
14
precision: 1.0
relevant_ground_truth {64, 33, 3, 37, 7, 72, 73, 48, 81, 18, 19, 58, 30, 63}
14
14
recall: 1.0


In [ ]:
import math
from collections import defaultdict, Counter
from abc import ABC, abstractmethod
from typing import List
import re
class VectorSpaceModel(RetrievalModel):
    def __init__(self):
        self.documents = []
        self.inverted_index = None
        self.inverted_index_filtered = None
        self.inverted_index_stemmed = None
        self.inverted_index_filtered_stemmed = None
        self.doc_term_freqs = []
        self.doc_vectors = []
        self.idf = {}
        self.idf_filtered = {}
        self.idf_stemmed = {}
        self.idf_filtered_stemmed = {}

    def build_inv_list(self, documents: List[Document], stopword_filtering=False, stemming=False):
        for document in documents:
            if stopword_filtering and stemming:
                terms = document.stemmed_filtered_terms
            elif stopword_filtering:
                terms = document.filtered_terms
            elif stemming:
                terms = document.stemmed_terms
            else:
                terms = document.terms
            self.documents.append(document)
            term_freq = Counter(terms)
            self.doc_term_freqs.append(term_freq)
            for term, freq in term_freq.items():
                if stopword_filtering and stemming:
                    self.inverted_index_filtered_stemmed[term].append((document.document_id, freq))
                elif stopword_filtering:
                    self.inverted_index_filtered[term].append((document.document_id, freq))
                elif stemming:
                    self.inverted_index_stemmed[term].append((document.document_id, freq))
                else:
                    self.inverted_index[term].append((document.document_id, freq))

    def calculate_idf(self, stopword_filtering=False, stemming=False):
        N = len(self.documents)
        if stopword_filtering and stemming:
            for term, postings in self.inverted_index_filtered_stemmed.items():
                df = len(postings)
                self.idf_filtered_stemmed[term] = math.log(N / df)
        elif stopword_filtering:
            for term, postings in self.inverted_index_filtered.items():
                df = len(postings)
                self.idf_filtered[term] = math.log(N / df)
        elif stemming:
            for term, postings in self.inverted_index_stemmed.items():
                df = len(postings)
                self.idf_stemmed[term] = math.log(N / df)
        else:
            for term, postings in self.inverted_index.items():
                df = len(postings)
                self.idf[term] = math.log(N / df)

    def document_to_representation(self, document: Document, stopword_filtering=False, stemming=False):
        term_freq = self.doc_term_freqs[document.document_id]
        if stopword_filtering and stemming:
            tf_idf_vector = {term: freq * self.idf_filtered_stemmed.get(term, 0) for term, freq in term_freq.items()}
        elif stopword_filtering:
            tf_idf_vector = {term: freq * self.idf_filtered.get(term, 0) for term, freq in term_freq.items()}
        elif stemming:
            tf_idf_vector = {term: freq * self.idf_stemmed.get(term, 0) for term, freq in term_freq.items()}
        else:
            tf_idf_vector = {term: freq * self.idf.get(term, 0) for term, freq in term_freq.items()}
        return tf_idf_vector

    def query_to_representation(self, query: str, stopword_filtering=False, stemming=False):
        query_terms = query.lower()
        query_terms = re.findall(r'\w+|[&|()-]', query_terms)
        term_freq = Counter(query_terms)
        if stopword_filtering and stemming:
            query_vector = {term: freq * self.idf_filtered_stemmed.get(term, 0) for term, freq in term_freq.items()}
        elif stopword_filtering:
            query_vector = {term: freq * self.idf_filtered.get(term, 0) for term, freq in term_freq.items()}
        elif stemming:
            query_vector = {term: freq * self.idf_stemmed.get(term, 0) for term, freq in term_freq.items()}
        else:
            query_vector = {term: freq * self.idf.get(term, 0) for term, freq in term_freq.items()}
        return query_vector

    def match(self, document_representation, query_representation) -> float:
        dot_product = sum(document_representation.get(term, 0) * weight for term, weight in query_representation.items())
        doc_norm = math.sqrt(sum(weight ** 2 for weight in document_representation.values()))
        query_norm = math.sqrt(sum(weight ** 2 for weight in query_representation.values()))
        if doc_norm == 0 or query_norm == 0:
            return 0.0
        return dot_product / (doc_norm * query_norm)

    def buckley_lewit_search(self, query: str, stemming: bool, stop_word_filtering: bool, gamma: int) -> list:
        if stop_word_filtering and stemming:
            if self.inverted_index_filtered_stemmed is None:
                self.inverted_index_filtered_stemmed = defaultdict(list)
                self.build_inv_list(fables, stop_word_filtering, stemming)
                self.calculate_idf(stopword_filtering=stop_word_filtering, stemming=stemming)
        elif stop_word_filtering:
            if self.inverted_index_filtered is None:
                self.inverted_index_filtered = defaultdict(list)
                self.build_inv_list(fables, stop_word_filtering, stemming)
                self.calculate_idf(stopword_filtering=stop_word_filtering, stemming=stemming)
        elif stemming:
            if self.inverted_index_stemmed is None:
                self.inverted_index_stemmed = defaultdict(list)
                self.build_inv_list(fables, stop_word_filtering, stemming)
                self.calculate_idf(stopword_filtering=stop_word_filtering, stemming=stemming)
        else:
            if self.inverted_index is None:
                self.inverted_index = defaultdict(list)
                self.build_inv_list(fables, stop_word_filtering, stemming)
                self.calculate_idf(stopword_filtering=stop_word_filtering, stemming=stemming)
        query = stem_query_terms(query)
        query_vector = self.query_to_representation(query, stopword_filtering=stop_word_filtering, stemming=stemming)
        scores = defaultdict(float)
        top_docs = {}

        # Sort query terms by descending weights
        sorted_query_terms = sorted(query_vector.items(), key=lambda item: item[1], reverse=True)

        for term, wqk in sorted_query_terms:
            if wqk <= 0:
                continue
            if stop_word_filtering and stemming:
                for doc_id, wdk in self.inverted_index_filtered_stemmed.get(term, []):
                    scores[doc_id] += wqk * wdk
                    self._insert_into_top_docs(top_docs, doc_id, scores[doc_id], gamma)
            elif stop_word_filtering:
                for doc_id, wdk in self.inverted_index_filtered.get(term, []):
                    scores[doc_id] += wqk * wdk
                    self._insert_into_top_docs(top_docs, doc_id, scores[doc_id], gamma)
            elif stemming:
                for doc_id, wdk in self.inverted_index_stemmed.get(term, []):
                    scores[doc_id] += wqk * wdk
                    self._insert_into_top_docs(top_docs, doc_id, scores[doc_id], gamma)
            else:
                for doc_id, wdk in self.inverted_index.get(term, []):
                    scores[doc_id] += wqk * wdk
                    self._insert_into_top_docs(top_docs, doc_id, scores[doc_id], gamma)

            if self._can_terminate_early(top_docs, query_vector, sorted_query_terms, gamma):
                break

        # Return the top gamma results
        return [(score, self.documents[doc_id]) for doc_id, score in sorted(top_docs.items(), key=lambda x: x[1], reverse=True)]

    def _insert_into_top_docs(self, top_docs, doc_id, score, gamma):
        if doc_id in top_docs:
            top_docs[doc_id] = max(top_docs[doc_id], score)
        else:
            top_docs[doc_id] = score
        if len(top_docs) > gamma + 1:
            lowest_doc_id = min(top_docs, key=top_docs.get)
            del top_docs[lowest_doc_id]

    def _can_terminate_early(self, top_docs, query_vector, sorted_query_terms, gamma):
        if len(top_docs) < gamma:
            return False

        current_threshold = sorted(top_docs.values(), reverse=True)[gamma - 1]
        max_remaining_weight = sum(wqk for term, wqk in sorted_query_terms if wqk > 0)
        return list(sorted(top_docs.values(), reverse=True))[gamma] + max_remaining_weight <= current_threshold

# Example usage
# doc1 = Document(0, "Doc1", "this is a sample document")
# doc2 = Document(1, "Doc2", "this document is another example")

vsm = VectorSpaceModel()
# fables should be defined as a list of Document objects
# for doc in fables:
#     vsm.add_document(doc)
# vsm.add_document(doc1)
# vsm.add_document(doc2)
# vsm.build_inv_list(fables, True, True)
# vsm.calculate_idf(stopword_filtering=True, stemming=True)

results = vsm.buckley_lewit_search("fox beast", stemming=True, stop_word_filtering=False, gamma=15)
for score, doc in results:
    print(score, doc)


12.373633423542962 D18: The Fox and the Stork("At one tim...")
11.278915979136393 D72: The Lion, the Fox, and the Beasts("The Lion o...")
10.942443742515179 D58: The Fox, the Cock, and the Dog("One moonli...")
10.605971505893967 D64: The Fox Without a Tail("It happene...")
10.605971505893967 D81: The Fox and the Goat("By an unlu...")
10.520670771351037 D23: The Bat, the Birds, and the Beasts("A great co...")
8.838309588244972 D73: The Ass's Brains("The Lion a...")
7.407119907217191 D33: The Fox and the Lion("When first...")
7.070647670595978 D07: The Fox and the Crow("A Fox once...")
7.070647670595978 D37: The Fox and the Cat("A Fox was ...")
7.070647670595978 D63: The Fox and the Mosquitoes("A Fox afte...")
5.639457989568196 D03: The Lion's Share("The Lion w...")
5.3029857529469835 D19: The Fox and the Mask("A Fox had ...")
3.535323835297989 D30: The Fox and the Grapes("One hot su...")
2.1041341542702074 D69: The Hare With Many Friends("A Hare was...")
2.1041341542702074 D70: The Lion